## Preprocessing data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# read data fram
data = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# create a column transformer
ct = make_column_transformer(
    (MinMaxScaler(), ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]),
    (OneHotEncoder(handle_unknown = "ignore"), ["ocean_proximity"])
)

# create X and y value
X = data.drop("median_house_value", axis = 1)
y = data["median_house_value"]

# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# fit the column tranformer to the training data
ct.fit(X_train)

# transform training and test data
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)


# total index yang terindikasi NaN
nan_index = len(np.argwhere(np.isnan(X_test_normal)))

# remove index which contains NaN
X_test_normal = X_test_normal[~np.ma.fix_invalid(X_test_normal).mask.any(axis=1)]
y_test = y_test[nan_index:]

In [ ]:
X_train_normal[0]

array([0.72908367, 0.01702128, 0.62745098, 0.0794547 , 0.09714463,
       0.06437961, 0.10228581, 0.19032151, 0.        , 0.        ,
       0.        , 0.        , 1.        ])

In [ ]:
X_test_normal[2]

array([0.23705179, 0.47021277, 0.2745098 , 0.07240958, 0.10350714,
       0.04324673, 0.0955435 , 0.15577716, 0.        , 0.        ,
       0.        , 0.        , 1.        ])

## Build a neural network

In [ ]:
# create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

# compile the model
model.compile(loss = tf.keras.losses.mae,
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
              metrics = ["mae"])

# fit the model
model.fit(X_train_normal, y_train, epochs = 100)

Epoch 1/100
516/516 [==============================] - 3s 2ms/step - loss: 75011.8281 - mae: 75011.8281
Epoch 2/100
516/516 [==============================] - 1s 2ms/step - loss: 50442.3320 - mae: 50442.3320
Epoch 3/100
516/516 [==============================] - 1s 2ms/step - loss: 49038.6250 - mae: 49038.6250
Epoch 4/100
516/516 [==============================] - 1s 2ms/step - loss: 48119.6133 - mae: 48119.6133
Epoch 5/100
516/516 [==============================] - 1s 2ms/step - loss: 47371.9492 - mae: 47371.9492
Epoch 6/100
516/516 [==============================] - 1s 2ms/step - loss: 46802.0312 - mae: 46802.0312
Epoch 7/100
516/516 [==============================] - 1s 2ms/step - loss: 46663.1836 - mae: 46663.1836
Epoch 8/100
516/516 [==============================] - 2s 3ms/step - loss: 46154.9648 - mae: 46154.9648
Epoch 9/100
516/516 [==============================] - 2s 3ms/step - loss: 45794.5469 - mae: 45794.5469
Epoch 10/100
516/516 [==============================] - 1s 2ms/s

In [ ]:
# evaluate the model
model.evaluate(X_test_normal, y_test)

123/123 [==============================] - 1s 4ms/step - loss: 36681.2773 - mae: 36681.2773


[36681.27734375, 36681.27734375]